In [ ]:
from msal import PublicClientApplication
import requests
import time

tenant_id = "<Entra_TenantID>"
client_id = "<Entra_ClientID>"

workspace_id = "<Fabric_WorkspaceID>"
lakehouse_id = "<Fabric_LakehouseID>"

# Script parameters

lakehouse_table_name = "<Sentiment_Table_Name>"
sourceColumnName = "<Sentiment_Source_ Column_Name>"
sentimentColumnName = "<Sentiment_Column_Name>"


In [ ]:
# Logging in to get a Token to access the API LivyApi

app = PublicClientApplication(
    client_id,
    authority="https://login.microsoftonline.com/43a26159-4e8e-442a-9f9c-cb7a13481d48"
)

result = None

# If no cached tokens or user interaction needed, acquire tokens interactively
if not result:
    result = app.acquire_token_interactive(scopes=["https://api.fabric.microsoft.com/Lakehouse.Execute.All", "https://api.fabric.microsoft.com/Lakehouse.Read.All", "https://api.fabric.microsoft.com/Item.ReadWrite.All", "https://api.fabric.microsoft.com/Workspace.ReadWrite.All", "https://api.fabric.microsoft.com/Code.AccessStorage.All", "https://api.fabric.microsoft.com/Code.AccessAzureKeyvault.All", 
    "https://api.fabric.microsoft.com/Code.AccessAzureDataExplorer.All", "https://api.fabric.microsoft.com/Code.AccessAzureDataLake.All", "https://api.fabric.microsoft.com/Code.AccessFabric.All"])

# Print the access token (you can use it to call APIs)
if "access_token" in result:
    print(f"Access token: {result['access_token']}")
else:
    print("Authentication failed or no access token obtained.")

if "access_token" in result:
    access_token = result['access_token']
    api_base_url ='https://api.fabric.microsoft.com/v1'
    livy_base_url = api_base_url + "/workspaces/"+workspace_id+"/lakehouses/"+lakehouse_id +"/livyApi/versions/2023-12-01/batches"
        headers = {"Authorization": "Bearer " + access_token}

In [ ]:
get_livy_get_batch = livy_base_url
get_batch_response = requests.get(get_livy_get_batch, headers = headers)
if get_batch_response.status_code == 200:
    print("API call successful")
    print(get_batch_response.json())
else:
    print(f"API call failed with status code: {get_batch_response.status_code}")
    print(get_batch_response.text)

In [ ]:
# submit payload to existing batch session

print('Submit a spark job via the livy batch API to ') 

newlakehouseName = "YourNewLakehouseName"
create_lakehouse = api_base_url + "/workspaces/" + workspace_id + "/items"
create_lakehouse_payload = {
    "displayName": newlakehouseName,
    "type": 'Lakehouse'
    }

create_lakehouse_response = requests.post(create_lakehouse, headers = headers, json = create_lakehouse_payload)
print(create_lakehouse_response.json())

payload_data = {
    "name":"livybatchdemo_with"+ newlakehouseName,
    "file":"abfss://YourABFSPathToYourPayload.py", 
    "conf": {
        "spark.targetLakehouse": "Fabric_LakehouseID",
        "spark.targetTable": targetTable,
        "spark.sourceColumnName": sourceColumnName,
        "spark.sentimentColumnName" sentimentColumnName
        }
    }

get_batch_response = requests.post(get_livy_get_batch, headers = headers, json = payload_data)

print("The Livy batch job submitted successful")
print(get_batch_response.json())